# A Gentle Start

Basic notions in machine learning include the following elements.
* The domain set is represented by $\mathcal{X}$.
* The label set is represented by $\mathcal{Y}$ and, at the onset, we take $\mathcal{Y} = \{ 0, 1 \}$.
* The training data is given by $S = \left\{ (x_i, y_i) \right\}$, where each element in this set belongs to $\mathcal{X} \times \mathcal{Y}$.


## Task

* The learner wishes to create a prediction rule by $h : \mathcal{X} \mapsto \mathcal{Y}$. This function is also called a predictor, a hypothesis, or a classifier.
* It is sometime useful to denote a (correct) labeling function $f : \mathcal{X} \mapsto \mathcal{Y}$ such that $y_i = f(x_i)$ for all $i$.


## Design Objective

* The last component needed is a measure of success. A typical such criterion can be derived from the probability of classification error:
\begin{equation*}
L_{(\mathcal{D}, f)} (h)
\doteq \Pr_{x \sim \mathcal{D}} (h(x) \neq f(x))
= \mathcal{D} \left( \{ x : h(x) \neq f(x) \} \right) .
\end{equation*}
In general, the nomenclature $L_{(\mathcal{D}, f)} (\cdot)$ refers to a loss function and $\mathcal{D}$ is the probability distribution underlying $\mathcal{X}$.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(100)

## Example 1.a

Let $\mathcal{D}$ be a multivariate normal distribution with location parameter $\boldsymbol{\mu} = \mathbf{0}$ and covariance matrix $\boldsymbol{\Sigma} = \mathbf{I}$.
Moreover, assume that the labeling function $f : \mathcal{X} \mapsto \mathbf{Y}$ is given by
\begin{equation*}
f(\mathbf{x}) = \operatorname{sign} \left( \langle \mathbf{x} | \mathbf{v} \rangle \right)
\end{equation*}
where $\mathbf{v}$ is a fixed vector.

In [ ]:
mu = 0
sigma = 1
fvector = np.random.normal(mu, sigma, (1,2))
print('Label Vector: ' + str(fvector))
X = np.random.normal(mu, sigma, (10,2))

def labelfunction(x_i):
    sign = np.sign(np.inner(x_i, fvector))
    return sign

XY_df = pd.DataFrame(X, columns=['pos1','pos2'])
XY_df['Y'] = labelfunction(X)
set1 = XY_df.loc[XY_df['Y'] > 0].as_matrix(['pos1','pos2'])
set0 = XY_df.loc[XY_df['Y'] < 0].as_matrix(['pos1','pos2'])

x, y = zip(*set1)
plt.scatter(x, y, c='b')
x, y = zip(*set0)
plt.scatter(x, y, c='r')
ax = plt.gca()
ax.set_xlim([-4,4])
ax.set_ylim([-4,4])
ax.grid()
plt.show()

# Empirical Risk Minimization (ERM)

Given that only the training sample is available to the learner, it may be appealing to search for a solution that works well on $S$:
\begin{equation*}
L_{S} (h) \doteq \frac{\left| \{ i \in [m] : h(x_i) \neq u_i \} \right|}{m} .
\end{equation*}
The objective function is often referred to as the empirical error and empirical risk.
Unfortunatly, ERM is prone to overfitting, a phenomenon where the hypothesis fits the training data well, but performs poorly in reality.


## Inductive Bias

A common strategy to prevent overfitting is to apply the ERM learning rule over a restricted hypothesis class $\mathcal{H}$.
The learner then uses the ERM rule to choose a predictor $h \in \mathcal{H}$, with the lowest possible error over $S$;
\begin{equation*}
\mathrm{ERM}_{\mathcal{H}} (S) \in \operatorname{argmin}_{h \in \mathcal{H}} L_S (h) .
\end{equation*}
Choosing a restricted hypothesis class can help protect against overfitting, but it may cause us a stronger inductive bias.


## Assumptions

**Realizability Assumption:** There exists $h^{\star} \in \mathcal{H}$ such that $L_{(\mathcal{D}, f)} (h^{\star}) = 0$.
Note that this assumption implies that $L_S(h^{\star}) = 0$ almost surely.

**IID Assumption:** The points in the training set $S$ are drawn independently according to the distribution $\mathcal{D}$. Consequently, $S \sim \mathcal{D}^m$ where $m$ is the cardinality of $S$.


## Example 1.b

Let $\mathcal{H}$ be the hypothesis class given by
\begin{equation*}
h_w(\mathbf{x}) = \operatorname{sign} \left( \langle \mathbf{x} | \mathbf{w} \rangle \right)
\end{equation*}
where $\mathbf{w}$ is a fixed vector.

## Probability Bounds

There is some probability that the sampled training data $S$ is nonrepresentative of the underlying distribution $\mathcal{D}$, in which case $L_{(\mathcal{D}, f)} (h_S)$ may feature bad performance.
The probability of getting a nonrepresentative sample is denoted by $\delta$, and $(1 − \delta)$ is call the confidence parameter of the prediction problem.

Let $\epsilon$ denote the accuracy parameter.
We interpret the event $L_{(\mathcal{D}, f)}(h_S) > \epsilon$ as a failure of the learner, whereas the condition $L_{(\mathcal{D}, f)}(h_S) \leq \epsilon$ is deemed an approximately correct predictor.
We wish to bound
\begin{equation*}
\mathcal{D}^m \left( \left\{ \left. S \right|_x : L_{(\mathcal{D}, f)}(h_S) > \epsilon \right\} \right)
\end{equation*}
Let $\mathcal{H}_{\mathrm{B}}$ be the set of bad hypothesis,
\begin{equation*}
\mathcal{H}_{\mathrm{B}}
= \left\{ h \in \mathcal{H} : L_{(\mathcal{D}, f)}(h) > \epsilon \right\} .
\end{equation*}
Also, let
\begin{equation*}
M = \left\{ \left. S \right|_x : \exists h \in \mathcal{H}_{\mathrm{B}}, L_S(h) = 0 \right\} .
\end{equation*}

The event $L_{\mathcal{D}, f}(h_S) > \epsilon$ can only happen if for some $h \in \mathcal{H}_{\mathrm{B}}$ we have $L_S (h) = 0$.
In other words, this event will only happen if our sample is in the set of misleading samples, $M$,
\begin{equation*}
\left\{ \left. S \right|_x :  L_{(\mathcal{D}, f)}(h_S) > \epsilon \right\} \subseteq M.
\end{equation*}
